In [225]:
import requests
from bs4 import BeautifulSoup
import json
from IPython.display import Markdown
import os
import re

kegg_ids_to_check = ["R09394", "R11372", "R09395", "R09726", "R09735"]
# search JSON of model if provided
file_path = '../rtRBA-C.json'
if os.path.exists(file_path):
    with open(file_path) as f:
        model = json.load(f)
        # update formulas to match MNX by removing lone 1s
        for m in model['metabolites']: 
            m['formula'] = re.sub(r'([A-Za-z])1\b', r'\1',re.sub(r'([A-Za-z])1([A-Za-z])', r'\1\2', m['formula']))
else:
    print(f"File not found: {file_path}")


In [10]:
'''queries for use at https://rdf.metanetx.org/:
# QUERY 1: Find if rxn is balanced
PREFIX mnx: <https://rdf.metanetx.org/schema/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX keggR: <https://identifiers.org/kegg.reaction:>

SELECT ?reaction (STR(?isBalanced) AS ?isBalancedValue)
WHERE {
    ?reaction a mnx:REAC .
    ?reaction mnx:reacXref keggR:R00703 .
    ?reaction mnx:isBalanced ?isBalanced .
}

# QUERY 2: Find rxn components
# URL format: https://rdf.metanetx.org/sparql/?default-graph-uri=https%3A%2F%2Frdf.metanetx.org%2F&query=PREFIX+mnx%3A+%3Chttps%3A%2F%2Frdf.metanetx.org%2Fschema%2F%3E%0D%0APREFIX+keggR%3A+%3Chttps%3A%2F%2Fidentifiers.org%2Fkegg.reaction%3A%3E%0D%0A%0D%0ASELECT+%3Fchem+%3Fchem_name+%3Fcomp+%3Fcomp_name+%3Fcoef+%3Fformula+%3Fcharge%0D%0AWHERE+%7B%0D%0A++++%3Freac+mnx%3AreacXref+keggR%3A{base_url}+.%0D%0A++++%3Freac+%3Fside+%3Fpart+.%0D%0A++++%3Fpart+mnx%3Achem+%3Fchem+%3B%0D%0A++++++++++mnx%3Acomp+%3Fcomp+%3B%0D%0A++++++++++mnx%3Acoef+%3Fc++++.%0D%0A++++%3Fchem+rdfs%3Acomment+%3Fchem_name+.%0D%0A++++%3Fcomp+rdfs%3Acomment+%3Fcomp_name+.%0D%0A++++OPTIONAL+%7B+%3Fchem+mnx%3Aformula+%3Fformula+%7D%0D%0A++++OPTIONAL+%7B+%3Fchem+mnx%3Acharge+%3Fcharge+%7D%0D%0A++++FILTER%28+%3Fside+IN+%28+mnx%3Aleft+%2C+mnx%3Aright+%29%29%0D%0A++++BIND%28+IF%28+%3Fside+%3D+mnx%3Aleft%2C+-+%3Fc%2C+%3Fc+%29+AS+%3Fcoef+%29%0D%0A%7D%0D%0A&format=text%2Fhtml
PREFIX mnx: <https://rdf.metanetx.org/schema/>
PREFIX keggR: <https://identifiers.org/kegg.reaction:>

SELECT ?chem ?chem_name ?comp ?comp_name ?coef ?formula ?charge
WHERE {
    ?reac mnx:reacXref keggR:R09394 .
    ?reac ?side ?part .
    ?part mnx:chem ?chem ;
          mnx:comp ?comp ;
          mnx:coef ?c    .
    ?chem rdfs:comment ?chem_name .
    ?comp rdfs:comment ?comp_name .
    OPTIONAL { ?chem mnx:formula ?formula }
    OPTIONAL { ?chem mnx:charge ?charge }
    FILTER( ?side IN ( mnx:left , mnx:right ))
    BIND( IF( ?side = mnx:left, - ?c, ?c ) AS ?coef )
}

# QUERY 2.1: find bigg ID too
# URL format: https://rdf.metanetx.org/sparql/?default-graph-uri=https%3A%2F%2Frdf.metanetx.org%2F&query=PREFIX+mnx%3A+%3Chttps%3A%2F%2Frdf.metanetx.org%2Fschema%2F%3E%0D%0APREFIX+keggR%3A+%3Chttps%3A%2F%2Fidentifiers.org%2Fkegg.reaction%3A%3E%0D%0A%0D%0ASELECT+%3Fchem+%3Fchem_name+%3Fcomp+%3Fcomp_name+%3Fcoef+%3Fformula+%3Fcharge%0D%0AWHERE+%7B%0D%0A++++%3Freac+mnx%3AreacXref+keggR%3A{base_url}+.%0D%0A++++%3Freac+%3Fside+%3Fpart+.%0D%0A++++%3Fpart+mnx%3Achem+%3Fchem+%3B%0D%0A++++++++++mnx%3Acomp+%3Fcomp+%3B%0D%0A++++++++++mnx%3Acoef+%3Fc++++.%0D%0A++++%3Fchem+rdfs%3Acomment+%3Fchem_name+.%0D%0A++++%3Fcomp+rdfs%3Acomment+%3Fcomp_name+.%0D%0A++++OPTIONAL+%7B+%3Fchem+mnx%3Aformula+%3Fformula+%7D%0D%0A++++OPTIONAL+%7B+%3Fchem+mnx%3Acharge+%3Fcharge+%7D%0D%0A++++FILTER%28+%3Fside+IN+%28+mnx%3Aleft+%2C+mnx%3Aright+%29%29%0D%0A++++BIND%28+IF%28+%3Fside+%3D+mnx%3Aleft%2C+-+%3Fc%2C+%3Fc+%29+AS+%3Fcoef+%29%0D%0A%7D%0D%0A&format=text%2Fhtml
PREFIX mnx: <https://rdf.metanetx.org/schema/>
PREFIX keggR: <https://identifiers.org/kegg.reaction:>

SELECT ?chem ?chem_name ?comp ?comp_name ?coef ?formula ?charge
WHERE {
    ?reac mnx:reacXref keggR:R09394 .
    ?reac ?side ?part .
    ?part mnx:chem ?chem ;
          mnx:comp ?comp ;
          mnx:coef ?c    .
    ?chem rdfs:comment ?chem_name .
    ?comp rdfs:comment ?comp_name .
    OPTIONAL { ?chem mnx:formula ?formula }
    OPTIONAL { ?chem mnx:charge ?charge }
    FILTER( ?side IN ( mnx:left , mnx:right ))
    BIND( IF( ?side = mnx:left, - ?c, ?c ) AS ?coef )
}

'''

'query for use at https://rdf.metanetx.org/:\nPREFIX mnx: <https://rdf.metanetx.org/schema/>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX keggR: <https://identifiers.org/kegg.reaction:>\n\nSELECT ?reaction (STR(?isBalanced) AS ?isBalancedValue)\nWHERE {\n    ?reaction a mnx:REAC .\n    ?reaction mnx:reacXref keggR:R00703 .\n    ?reaction mnx:isBalanced ?isBalanced .\n}\n'

In [243]:
output_dicts = []
unique_mets = set() # for all mets, including ones that may already exist
unique_new_mets = set()

def check_reaction_balance(reaction_id):
    base_url = "https://rdf.metanetx.org/sparql/?default-graph-uri=https%3A%2F%2Frdf.metanetx.org%2F&format=text%2Fhtml"

    query_url = (
        f"{base_url}&query=PREFIX+mnx%3A+%3Chttps%3A%2F%2Frdf.metanetx.org%2Fschema%2F%3E%0D%0A"
        "PREFIX+owl%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0D%0A"
        "PREFIX+rdf%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23%3E%0D%0A"
        "PREFIX+rdfs%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0D%0A"
        "PREFIX+keggR%3A+%3Chttps%3A%2F%2Fidentifiers.org%2Fkegg.reaction%3A%3E%0D%0A%0D%0A"
        f"SELECT+%28STR%28%3FisBalanced%29+AS+%3FisBalancedValue%29+%3Freaction%0D%0A"
        "WHERE+%7B%0D%0A"
        "++++%3Freaction+a+mnx%3AREAC+.%0D%0A"
        f"++++%3Freaction+mnx%3AreacXref+keggR%3A{reaction_id}+.%0D%0A"
        "++++%3Freaction+mnx%3AisBalanced+%3FisBalanced+.%0D%0A"
        "%7D%0D%0A"
    )

    response = requests.get(query_url)
    
    if response.ok:
        soup = BeautifulSoup(response.text, 'html.parser')
        is_balanced_value = soup.find('pre').text.strip()
        reaction_url = soup.find('a', href=lambda href: href and href.startswith('https://rdf.metanetx.org/reac/'))
        
        if reaction_url:
            reaction_url = reaction_url['href']
        else:
            reaction_url = None
        display(Markdown(f"[{reaction_id} is mass- & charge-balanced: {is_balanced_value}]({reaction_url})"))
        # print(f"Reaction {reaction_id} is mass- & charge-balanced: {is_balanced_value}")
        # print(f"Reaction URL: {reaction_url}")

        # if is_balanced_value.lower() != "true":
            #print(f"Fetching additional information for reaction {reaction_id}")
        fetch_and_check_balance(reaction_id, reaction_url)
    else:
        print(f"Error querying for reaction {reaction_id}")

def fetch_and_check_balance(reaction_id, reaction_url):
    base_url = (f"https://rdf.metanetx.org/sparql/?default-graph-uri=https%3A%2F%2Frdf.metanetx.org%2F&query=PREFIX+mnx%3A+%3Chttps%3A%2F%2Frdf.metanetx.org%2Fschema%2F%3E%0D%0A"
                "PREFIX+keggR%3A+%3Chttps%3A%2F%2Fidentifiers.org%2Fkegg.reaction%3A%3E%0D%0A%0D%0A"
                f"SELECT+%3Fchem+%3Fchem_name+%3Fcomp+%3Fcomp_name+%3Fcoef+%3Fformula+%3Fcharge%0D%0A"
                "WHERE+%7B%0D%0A"
                f"++++%3Freac+mnx%3AreacXref+keggR%3A{reaction_id}+.%0D%0A"
                "++++%3Freac+%3Fside+%3Fpart+.%0D%0A"
                "++++%3Fpart+mnx%3Achem+%3Fchem+%3B%0D%0A"
                "++++++++++mnx%3Acomp+%3Fcomp+%3B%0D%0A"
                "++++++++++mnx%3Acoef+%3Fc++++.%0D%0A"
                "++++%3Fchem+rdfs%3Acomment+%3Fchem_name+.%0D%0A"
                "++++%3Fcomp+rdfs%3Acomment+%3Fcomp_name+.%0D%0A"
                "++++OPTIONAL+%7B+%3Fchem+mnx%3Aformula+%3Fformula+%7D%0D%0A"
                "++++OPTIONAL+%7B+%3Fchem+mnx%3Acharge+%3Fcharge+%7D%0D%0A"
                "++++FILTER%28+%3Fside+IN+%28+mnx%3Aleft+%2C+mnx%3Aright+%29+%29%0D%0A"
                "++++BIND%28+IF%28+%3Fside+%3D+mnx%3Aleft%2C+-+%3Fc%2C+%3Fc+%29+AS+%3Fcoef+%29%0D%0A"
                "%7D%0D%0A&format=text%2Fhtml"
                )

    response = requests.get(base_url)
    
    if response.ok:
        soup = BeautifulSoup(response.text, 'html.parser')
        rows = soup.find_all('tr')[1:]  # Skip the header row

        charge_balance = 0
        element_changes = {}
        missing_info = []
        metabolites = {}
        for row in rows:
            columns = row.find_all('td')
            chem = columns[0].text.strip()
            chem_name = columns[1].text.strip()
            coef = float(columns[4].text.strip())
            formula = columns[5].text.strip() if columns[5].text.strip() != "None" else None
            charge_str = columns[6].text.strip()
            metabolites[chem] = {
                "formula": formula,
                "charge_str": charge_str,
                "coef": coef,
                "chem_name": chem_name.strip('"'),
                "chem": chem
            }
            # Checks metabolites in model for a match
            for chem, metabolite in metabolites.items():
                chem_name = metabolite['chem_name']
                charge_str = metabolite['charge_str']
                formula = metabolite['formula']

                # for m in model['metabolites']:
                # print(m['id'])

                if charge_str != "None" and charge_str != "":
                    charge = float(charge_str)
                else:
                    charge = 0
                # matches = [m for m in model['metabolites'] if m['name'] == chem_name and float(m['charge']) == float(charge) and m['formula'] == formula.replace('"','')]
                matches = []
                best_matches = []
                for m in model['metabolites']:
                    if float(m['charge']) == float(charge) and m['formula'] == formula.replace('"',''):
                        if m['name'] == chem_name:
                            best_matches.append(m)
                        matches.append(m)

                met_output_dict = {
                "id":chem.replace('"',''),
                "name":chem_name.replace('"',''),
                "compartment":"",
                "charge":charge_str.replace('"',''),
                "formula":formula.replace('"',''),
                "notes":{
                "formula_charge_source":"auto",
                },
                "annotation":{
                "chebi":"CHEBI:",
                "kegg.compound":"",
                "pubchem.compound":"",
                "sbo":"",
                "seed.compound":""
                }
                }

                if best_matches:
                    if len(best_matches) == 1:
                        metabolites[chem]['printed_name'] = best_matches[0]['id']
                    elif len(best_matches) > 1:
                        metabolites[chem]['printed_name'] = ', '.join([chem] + [m['id'] for m in best_matches])
                    # print(matches[0]['id'])
                else:
                    if len(matches) > 1:
                        metabolites[chem]['printed_name'] = ', '.join(['Ambiguous: ',chem] + [m['id'] for m in matches])
                    elif len(matches) == 1:
                        metabolites[chem]['printed_name'] = 'Ambiguous: ' + matches[0]['id']
                    else:
                        metabolites[chem]['printed_name'] = chem
                        # add metabolite template if it doesn't exist in model
                    unique_new_mets.add(json.dumps(met_output_dict, indent=2))

            charge_balance += coef * charge if charge else 0

            if not formula:
                #print(f"Missing formula for {chem}")
                missing_info.append(f"Component: {chem}, Formula: {formula}, Charge: {charge}")
            else:
                # Extract elements and their coefficients from the formula
                element_info = extract_elements_from_formula(formula)
                if element_info is not None:  # Check if element_info is not None
                    for element, element_coef in element_info.items():
                        element_changes[element] = element_changes.get(element, 0) + coef * element_coef
        if charge_balance != 0:
            display(Markdown(f"[{reaction_id} charge balance (products - reactants): {charge_balance}]({base_url})"))
        # print(f"Charge Balance: {charge_balance}")
        # Reports mass balance issues if any exist.
        if element_changes:
            if any(change != 0 for change in element_changes.values()):
                print("Element Changes:")
                for element, change in element_changes.items():
                    if change != 0:
                        print(f"{element}: {change}")

        if missing_info:
            print("Components with missing information:")
            for info in missing_info:
                print(info)

        kegg_reaction_id = reaction_id
        metanetx_reaction_id = reaction_url.rsplit('/', 1)[1]

        output_dict = {
            "id": reaction_id,
            "name": reaction_id,
            "lower_bound": -1000.0,
            "upper_bound": 1000.0,
            "metabolites": "",
            "gene_reaction_rule": "",
            "subsystem": "Unassigned",
            "annotation": {
                "ec-code": "",
                "kegg.reaction": kegg_id,
                "metanetx.reaction": metanetx_reaction_id,
                "sbo": ""
            },
            }
        output_dict["metabolites"] = {metabolite["printed_name"]: metabolite["coef"] for metabolite in metabolites.values()}
        

            # for chem, coef in metabolites.items():
            #     formula = chem
            #     chem_url = f"https://example.com/chem/{chem}"  # Replace with the actual URL
            #     print("\"metabolites\"][\"{}\"][\"chem\": \"{}\"".format(formula, chem))
            #     print("\"metabolites\"][\"{}\"][\"link\": \"[{}]({})\"".format(formula, formula, chem_url))
            #     print("\"metabolites\"][\"{}\"][\"coef\": {}".format(formula, coef))
            # print("{")
            # print("\"id\": \"{}\"".format(reaction_id)+",")
            # print("\"name\": \"{}\"".format(reaction_id)+",")
            # print("\"metabolites\": {")
            # for chem, coef in metabolites.items():
            #     display(Markdown(f"[\"{chem}\": {coef}]({chem})"))
            # print("}")
            # print("\"lower_bound\": -1000.0,")
        # print("\"upper_bound\": 1000.0,")
        # print("\"gene_reaction_rule\": \"\",")
        # print("\"subsystem\": \"Unassigned\",")
        # print("\"annotation\": {")
        # print("\"ec-code\": \"\",")
        # print("\"kegg.reaction\": \"{}\"".format(kegg_id)+",")
        # print("\"metanetx.reaction\": \"{}\"".format(metanetx_reaction_id)+",")
        # print("\"sbo\": \"\"")
        # print("}"+f"\n"+"}")

        # print output_dict
        # print(json.dumps(output_dict, indent=2))
        output_dicts.append(output_dict)
    else:
        print(f"Error fetching additional information for reaction {reaction_id}")

def extract_elements_from_formula(formula):
    elements = {}
    # list of periodic table element symbols (from https://pubchem.ncbi.nlm.nih.gov/periodic-table/)
    # ["H","He","Li","Be","B","C","N","O","F","Ne","Na","Mg","Al","Si","P","S","Cl","Ar","K","Ca","Sc","Ti","V","Cr","Mn","Fe","Co","Ni","Cu","Zn","Ga","Ge","As","Se","Br","Kr","Rb","Sr","Y","Zr","Nb","Mo","Tc","Ru","Rh","Pd","Ag","Cd","In","Sn","Sb","Te","I","Xe","Cs","Ba","La","Ce","Pr","Nd","Pm","Sm","Eu","Gd","Tb","Dy","Ho","Er","Tm","Yb","Lu","Hf","Ta","W","Re","Os","Ir","Pt","Au","Hg","Tl","Pb","Bi","Po","At","Rn","Fr","Ra","Ac","Th","Pa","U","Np","Pu","Am","Cm","Bk","Cf","Es","Fm","Md","No","Lr","Rf","Db","Sg","Bh","Hs","Mt","Ds","Rg","Cn","Nh","Fl","Mc","Lv","Ts","Og"]
    current_element = ""
    current_coef = ""
    i = 0

    while i < len(formula):
        if formula[i].isalpha():
            current_element = formula[i]
            i += 1
            while i < len(formula) and formula[i].isalpha() and current_element in elements:
                current_element += formula[i]
                i += 1
            while i < len(formula) and formula[i].isdigit():
                current_coef += formula[i]
                i += 1
            coef = int(current_coef) if current_coef else 1
            elements[current_element] = coef
            current_element = ""
            current_coef = ""
        else:
            i += 1
    return elements

if __name__ == "__main__":
    for kegg_id in kegg_ids_to_check:
        check_reaction_balance(kegg_id)
    display("all rxns in JSON form:")
    for reaction in output_dicts:
        print(json.dumps(reaction, indent=4)+",")
    display("new mets in JSON form:")
    for met in unique_new_mets:
        print(met+",")

[R09394 is mass- & charge-balanced: false](https://rdf.metanetx.org/reac/MNXR112822)

[R09394 charge balance (products - reactants): 1.0](https://rdf.metanetx.org/sparql/?default-graph-uri=https%3A%2F%2Frdf.metanetx.org%2F&query=PREFIX+mnx%3A+%3Chttps%3A%2F%2Frdf.metanetx.org%2Fschema%2F%3E%0D%0APREFIX+keggR%3A+%3Chttps%3A%2F%2Fidentifiers.org%2Fkegg.reaction%3A%3E%0D%0A%0D%0ASELECT+%3Fchem+%3Fchem_name+%3Fcomp+%3Fcomp_name+%3Fcoef+%3Fformula+%3Fcharge%0D%0AWHERE+%7B%0D%0A++++%3Freac+mnx%3AreacXref+keggR%3AR09394+.%0D%0A++++%3Freac+%3Fside+%3Fpart+.%0D%0A++++%3Fpart+mnx%3Achem+%3Fchem+%3B%0D%0A++++++++++mnx%3Acomp+%3Fcomp+%3B%0D%0A++++++++++mnx%3Acoef+%3Fc++++.%0D%0A++++%3Fchem+rdfs%3Acomment+%3Fchem_name+.%0D%0A++++%3Fcomp+rdfs%3Acomment+%3Fcomp_name+.%0D%0A++++OPTIONAL+%7B+%3Fchem+mnx%3Aformula+%3Fformula+%7D%0D%0A++++OPTIONAL+%7B+%3Fchem+mnx%3Acharge+%3Fcharge+%7D%0D%0A++++FILTER%28+%3Fside+IN+%28+mnx%3Aleft+%2C+mnx%3Aright+%29+%29%0D%0A++++BIND%28+IF%28+%3Fside+%3D+mnx%3Aleft%2C+-+%3Fc%2C+%3Fc+%29+AS+%3Fcoef+%29%0D%0A%7D%0D%0A&format=text%2Fhtml)

Element Changes:
H: -1.0
Components with missing information:
Component: https://rdf.metanetx.org/chem/MNXM1102419, Formula: , Charge: 0
Component: https://rdf.metanetx.org/chem/MNXM8975, Formula: , Charge: 0


[R11372 is mass- & charge-balanced: true](https://rdf.metanetx.org/reac/MNXR114669)

[R09395 is mass- & charge-balanced: false](https://rdf.metanetx.org/reac/MNXR143670)

...

[R09735 is mass- & charge-balanced: true](https://rdf.metanetx.org/reac/MNXR151954)

'all rxns in JSON form:'

{
    "id": "R09394",
    "name": "R09394",
    "lower_bound": -1000.0,
    "upper_bound": 1000.0,
    "metabolites": {
        "Ambiguous: , https://rdf.metanetx.org/chem/MNXM1100966, gtp_c, gtp_m": -1.0,
        "https://rdf.metanetx.org/chem/MNXM738804, met__L_c, met__L_e, met__L_m": -1.0,
        "Ambiguous: , https://rdf.metanetx.org/chem/MNXM1102072, amet_c, amet_e, amet_m, amet_rm": 1.0,
        "Ambiguous: , https://rdf.metanetx.org/chem/MNXM1103313, dad_2_c, dad_2_e, dad_5_c, dad_5_m": -1.0,
        "https://rdf.metanetx.org/chem/MNXM1103553, gtp_c, gtp_m": 1.0,
        "Ambiguous: , https://rdf.metanetx.org/chem/MNXM1102419, CATION1_en, CATION1_vm, CATION1_gm, CATION1_mm, CATION1_rm, CATION1_v, CATION1_x, CATION1_n, CATION1_m, CATION1_l, CATION1_g, CATION1_c, CATION1_r, CATION1_e, metal2_c, metal2_e, metal2_m, metal2_x, feCATION_c, feCATION_e, feCATION_m, feCATION_n, mg2ormn2_c, mg2ormn2_e, mg2ormn2_en, mg2ormn2_m, mg2ormn2_vm, heme_c, heme_m, heme_r, heme_x": 1.0,
        "A

'new mets in JSON form:'

{
  "id": "https://rdf.metanetx.org/chem/MNXM1102072",
  "name": "(S)-S-adenosyl-L-methionine",
  "compartment": "",
  "charge": "1",
  "formula": "C15H23N6O5S",
  "notes": {
    "formula_charge_source": "auto"
  },
  "annotation": {
    "chebi": "CHEBI:",
    "kegg.compound": "",
    "pubchem.compound": "",
    "sbo": "",
    "seed.compound": ""
  }
},
{
  "id": "https://rdf.metanetx.org/chem/MNXM1103313",
  "name": "5'-deoxyadenosine",
  "compartment": "",
  "charge": "0",
  "formula": "C10H13N5O3",
  "notes": {
    "formula_charge_source": "auto"
  },
  "annotation": {
    "chebi": "CHEBI:",
    "kegg.compound": "",
    "pubchem.compound": "",
    "sbo": "",
    "seed.compound": ""
  }
},
{
  "id": "https://rdf.metanetx.org/chem/MNXM726090",
  "name": "molybdic acid",
  "compartment": "",
  "charge": "0",
  "formula": "H2MoO4",
  "notes": {
    "formula_charge_source": "auto"
  },
  "annotation": {
    "chebi": "CHEBI:",
    "kegg.compound": "",
    "pubchem.compound": "",
    "sbo"